In [ ]:
import numpy as np
import pandas as pd
import scipy.io as io
import sklearn
import seaborn as sns
import time
from tqdm import tqdm
import os
import pickle

import paddle
import paddle.nn as nn
import paddle.nn.functional as F
from paddle.io import Dataset
from paddle.io import DataLoader
from paddle.io import TensorDataset


In [ ]:
def Normalization(x):
    xx=[]
    for i in x:
        xx.append(np.array((i - i.mean()) / i.std()))
    return np.array(xx)

def data_split(mat, test_size = 0.25, use_normal = False,decline_clas=False, verbose=1):
    datav = mat['vib']
    datac = mat['cur2']
    data = np.concatenate((datav,datac),axis=1)
    y = mat['label']  
    if decline_clas:
        data = data[:480*8]
        y = y[:480*8]

    if use_normal:
        data = Normalization(data)
    data = np.expand_dims(data,axis=1)
    x_train, x_test,y_train, y_test = train_test_split(data,y, 
                    test_size = test_size, random_state = 1314, shuffle = True ,stratify=y)
    x_train = paddle.cast(paddle.to_tensor(x_train), dtype='float32')
    y_train = (paddle.cast(paddle.to_tensor(y_train), dtype='int64')) 
    x_test = paddle.cast(paddle.to_tensor(x_test), dtype='float32')
    y_test = (paddle.cast(paddle.to_tensor(y_test), dtype='int64')) 
    train_Dataset = TensorDataset([paddle.to_tensor(x_train), paddle.to_tensor(y_train)])
    test_Dataset = TensorDataset([paddle.to_tensor(x_test), paddle.to_tensor(y_test)])
    if verbose==1:
        print('train example: ',x_train.shape,'\n','test example: ',x_test.shape)
    return train_Dataset, test_Dataset

In [ ]:
train_dataset, test_dataset = data_split(mat4, test_size = 0.25, use_normal = False, decline_clas=False, verbose=1)